In [ ]:
from main.utils.analysis import *
%matplotlib inline

In [ ]:
params = {'data_pth':'../../1/mj2非银版/data/modeling_data.csv',
    'date_colName': 'apply_date',
    'y_colName': 'credit_target',
    'org_colName':'org',
    'data_encode':'utf-8',
    'key_colNames' : ['mobile_org', 'apply_date', 'org']}
data = get_dataset(**params)

## 1. 数据分析(EDA)

### 1.1 去除异常月份

In [ ]:
params = {
    'data': data,
    'minYmBadsample': 10,
    'minYmSample': 1000
}
data = drop_abnormal_ym(**params)

### 1.2 机构逐月坏样率概览

In [ ]:
datasetStatis = org_analysis(data)
display(datasetStatis.head(2))
datasetStatis.to_csv("process results/datasetStatis.csv", index=False)

### 1.3 变量分机构和总体坏样率概览

In [ ]:
channel = {'银行': ['字节放心借', '滴滴金融'], '非银': ['xk_hc','fd_hc', '长银消金', '久恒融担（恒昌）', '分期乐欺诈',
       '宁夏海胜通'], '整体':list(data.new_org.unique())}

In [ ]:
miss_org, miss_channel = missing_check(**{'data': data, 'channel': channel})
display(miss_org.head(1), miss_channel.head(1))
miss_org.to_csv('process results/miss_org.csv', index=False)
miss_channel.to_csv('process results/miss_channel.csv', index=False)

### 1.4 变量分机构和渠道psi概览

In [ ]:
res_psi_org, res_psi_channel = detect_psi(**{'data': data, 'channel': channel})
res_psi_org.to_csv("process results/psi_org.csv", index=False)
res_psi_channel.to_csv('process results/psi_channel.csv', index=False)
display(res_psi_org.head(4), res_psi_channel.head(4))

### 1.5 变量分机构和渠道和总体iv概览

In [ ]:
res_iv_org, res_iv_channel = detect_iv(**{'data':data, 'method':'quantile', 'bins':5, 'channel': channel})
res_iv_org.to_csv('process results/iv_org.csv', index=False)
res_iv_channel.to_csv("process results/iv_channel.csv", index=False)
display(res_iv_org.head(2), res_iv_channel.head(2))

### 1.6 变量总体相似性概览

In [ ]:
indices, corr = detect_correlation(**{'data':data, 'method':'pearson', 'max_corr':0.85})
corr.to_csv('process results/correlation.csv', index=False)

## 2. 变量筛选 (iv psi correlation null_importance forward_selection)

### 2.1 去除高缺失率变量

In [ ]:
data = drop_highmiss_features(**{'data':data, 'ratio': 0.3, 'cnt': 3, 'miss_org': miss_org, 'miss_channel': miss_channel})

### 2.2 去除高psi变量

In [ ]:
data = drop_highpsi_features(**{'data':data, 'res_psi_org':res_psi_org, 'cnt':2, 'ratio':0.1})

### 2.3 去除低iv变量

In [ ]:
data = drop_lowiv_features(**{'data':data, 'miniv':0.1, 'cnt':5, 'res_iv_org':res_iv_org, 'res_iv_channel': res_iv_channel})

### 2.4 去除高相似性变量

In [ ]:
data = drop_highcorrelation_features(**{'data':data, 'res_iv_channel':res_iv_channel,'indices':indices, 'channel':'整体'})

### 2.5 null importance去除高噪音变量

In [ ]:
data = drop_highnoise_features(**{'data': data, 'n_estimators':100, 'max_depth': 5})

### 2.6 前向特征选择 (optional)